In [ ]:
from deep_translator import GoogleTranslator
import spacy
import services.influencers_sevice as influencers_service
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
import json 
import glob
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
#visualisation 
import pyLDAvis
import pyLDAvis.gensim

In [ ]:
translator = GoogleTranslator(source='auto', target='en')
influencers = influencers_service.get_influencers()
posts=influencers_service.get_posts()
stopwords_en = stopwords.words('english')

In [ ]:
def translate_post(posts,key):
    for post in posts:
        content = post.get(key)
        print(content)
        try:
            translated_content = translator.translate(content)
            print("after tanslation", translated_content)
            influencers_service.update_post(post, key,translated_content)
    
        except Exception as e:
            print(f"An exception occurred: {e}. Skipping this caption.")

In [ ]:
def lemmatize_text(text):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    allowed_postags = ["NOUN", "AD", "VERB", "ADV"]
    doc = nlp(text)
    new_text = []
    for word in doc:
        print(word)
        new_text.append(word.lemma_)
    lemmatized_text = " ".join(new_text)
    return lemmatized_text

In [ ]:
def lemmatize_post(posts,key):
    for post in posts: 
        try:
            content = post.get(key)
            lemmatized_content = lemmatize_text(content)
            influencers_service.update_post(post, key , lemmatized_content)
        except Exception as e:
            print(f"An exception occurred: {e}. Skipping this hashtag.")

In [ ]:
def remove_emojis(text):
    if(text):
        print("before emoji removal :",text)
        return unidecode(text)
    return text


In [ ]:
def remove_post_emojis(posts,key):
    for post in posts:
        content = post.get(key)
        print(content)
        no_emoji_content = remove_emojis(content)
        influencers_service.update_post(post, key, no_emoji_content)

In [ ]:
def remove_stopwords(text, stopwords_list):
    words = text.split()
    words = [word for word in words if word not in stopwords_list]
    return ' '.join(words)

In [ ]:
def remove_post_stopwords(posts,key):
    for post in posts:
        content = post.get(key)
        try:
            no_stopwords_content = remove_stopwords(content, stopwords_en)
            print(no_stopwords_content)
            influencers_service.update_post(post, key, no_stopwords_content)
        except Exception as e:
            print(f"An exception occurred: {e}. Skipping this post")

In [ ]:
# translate_post(posts, 'caption')

In [ ]:
# remove_post_emojis(posts, 'caption')

In [ ]:
# remove_post_stopwords(posts, 'caption')

In [ ]:
# lemmatize_post(posts, 'caption')

In [ ]:
# print("doooooooooone")